In [ ]:
import os

import rawpy as rp
import exiv2
from src.RawHandler.RawHandler import RawHandler
from src.RawHandler.RawHandlerRawpy import RawHandlerRawpy

import matplotlib.pyplot as plt
from colour_demosaicing import demosaicing_CFA_Bayer_Malvar2004


In [ ]:
import rawpy

In [ ]:
path = "/Volumes/T7/example_raw/"
files = [path+f for f  in os.listdir(path) if not '._' in f]
files = [f for f in files if not 'xmp' in f]

files

In [ ]:
# rh = RawHandlerRawpy( '/Volumes/T7/example_raw/DSCF5876.RAF')
colorspace='camera'
# rh = RawHandler( '/Volumes/T7/example_raw/793A6384.CR2', colorspace=colorspace)
rh2 = RawHandlerRawpy('/Volumes/T7/example_raw/793A6384.CR2')

# rh = RawHandler( '/Volumes/EasyStore/RAWNIND/Bayer_TEST_MuseeL-bluebirds-A7C_ISO65535_sha1=2d31480808a4068d0f6c17a9204cb375f35b3a78.arw', colorspace=colorspace)

rh2 = RawHandlerRawpy( '/Volumes/EasyStore/RAWNIND/Bayer_TEST_MuseeL-bluebirds-A7C_ISO65535_sha1=2d31480808a4068d0f6c17a9204cb375f35b3a78.arw')

# rh2 = RawHandlerRawpy( '/Volumes/T7/example_raw/DSCF5760.RAF')
# rh2 = RawHandlerRawpy( '/Volumes/T7/RAWNIND/Bayer_TEST_MuseeL-bluebirds-A7C_ISO125_sha1=4a6064cb0dfffb671c3ee38af56d0751929796bb.arw')
# rh2 = RawHandlerRawpy( '/Volumes/T7/example_raw/DSCF0319.RAF')


#rawforge test_run_baseline.pt /Volumes/EasyStore/test_denoise/Bayer_TEST_MuseeL-bluebirds-A7C_ISO65535_sha1=2d31480808a4068d0f6c17a9204cb375f35b3a78.arw baseline.dng


In [ ]:
rh2.to_dng(f"test_resave.dng", uint_img=rh2.rawpy_object.raw_image_visible[0:600, 0:600])

In [ ]:
img = rh2.rawpy_object.postprocess(
            user_wb=[1, 1, 1, 1],
            output_color=rawpy.ColorSpace.raw,
            demosaic_algorithm= rawpy.DemosaicAlgorithm(3),
            no_auto_bright=True,
            use_camera_wb=False,
            use_auto_wb=False,
            gamma=(1, 1),
            user_flip=0,
            output_bps=16,
            no_auto_scale=True,
        ) / rh2.rawpy_object.white_level
img = img.transpose(2, 0, 1)

In [ ]:
img.shape

In [ ]:
plt.imshow(img[:, 3000:3000+1024,3400:3400+1024].transpose(1, 2, 0)*2)

In [ ]:
model_path = "/Volumes/EasyStore/Downloads/test_run_baseline_camera.pt"
model_path = "/Volumes/EasyStore/Downloads/test_run_baseline.pt"
model_path = "/Volumes/EasyStore/Downloads/xtrans_fixed_exposure_500.pt"

import torch

In [ ]:
loaded = torch.jit.load(model_path, map_location='cpu')

In [ ]:
model = loaded.eval().to('mps')

In [ ]:
x = 1000
y = 3000
x = 2598
y = 3000

# x = 3000
# y = 3400
# x = 0
# y = 0
size = 1020
name = "xtrans_fixed_exposure_500_15_chart.dng"


_img = img[:, x:x+size, y:y+size]

_img = torch.tensor(_img).float().to('mps').unsqueeze(0).to('mps')
conditioning = torch.tensor([[rh2.full_metadata.get_ISO()]]).to('mps')/6400
_, mask = rh2.compute_mask_and_sparse(dims=(x, size+x, y, size+y))
mask = torch.tensor(mask).bool().to('mps').unsqueeze(0).to('mps')

conditioning = torch.tensor([[65535]]).to('mps')/6400*1
conditioning = torch.tensor([[1]]).to('mps')*15

In [ ]:
rh2.to_dng(f"{name}", uint_img=rh2.rawpy_object.raw_image_visible[x:x+size, y:y+size])

In [ ]:
rh2.full_metadata.get_ISO()

In [ ]:
with torch.no_grad():
    cfa = model(_img, conditioning)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
plt.imshow(cfa.cpu()[0].permute(1, 2, 0)**.5*2)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
plt.imshow((_img-cfa).cpu()[0].permute(1, 2, 0)*4+0.5)

In [ ]:
# rgb_transform = rh.rgb_colorspace_transform()
# import numpy as np
# inv = np.linalg.inv(rgb_transform)

In [ ]:
cfa = cfa[0].cpu().numpy()

In [ ]:
# inv = np.linalg.inv(rh.rgb_colorspace_transform())
# inv

In [ ]:

# orig_dims = cfa.shape
# cfa = (inv @ cfa.reshape(3, -1)).reshape(orig_dims)


In [ ]:
cfa = cfa.clip(0, 1)

In [ ]:
plt.imshow(cfa.transpose(1, 2, 0)**.5)

In [ ]:
cfa.shape, mask.shape

In [ ]:
cfa[~mask[0].cpu().numpy()] = 0

In [ ]:
plt.imshow(cfa.transpose(1, 2, 0)**.5*2)

In [ ]:
cfa = cfa.sum(axis=0)
# org_cfa = sparse[0].sum(axis=0).cpu().numpy()

In [ ]:
rh2.core_metadata.black_level_per_channel[0], rh2.core_metadata.white_level

In [ ]:
cfa = cfa * ( rh2.core_metadata.white_level) + rh2.core_metadata.black_level_per_channel[0]

In [ ]:
cfa.mean()

In [ ]:
rh2.core_metadata.black_level_per_channel[0]

In [ ]:
rh2.rawpy_object.black_level_per_channel

In [ ]:
cfa.shape

In [ ]:
rh2.to_dng(f"denoise{name}", uint_img=cfa, user_black_level=0)

In [ ]:
rp = rawpy.imread("denoise{name}")

In [ ]:
rp.raw_image_visible

In [ ]:
rp.black_level_per_channel

In [ ]:
rh2.core_metadata.black_level_per_channel[0]

In [ ]:
org_cfa *= rh.core_metadata.white_level
rh.to_dng(f"{name}", uint_img=org_cfa)

In [ ]:
from debayer import Debayer5x5
from debayer import Layout

In [ ]:
f = Debayer5x5(layout=Layout.RGGB).to('mps')

In [ ]:
cfa = torch.tensor(cfa).unsqueeze(0).unsqueeze(0).to('mps')/rh.core_metadata.white_level
org_cfa = torch.tensor(org_cfa).unsqueeze(0).unsqueeze(0).to('mps')/rh.core_metadata.white_level

In [ ]:
cfa.shape

In [ ]:
cfa2 = torch.cat([cfa, org_cfa], axis=0)
cfa2.shape

In [ ]:
output = f(cfa2)

In [ ]:
rh.core_metadata.black_level_per_channel[0]

In [ ]:
output[0].mean(), output[1].mean()

In [ ]:
plt.imshow((output[0]-output[1]).cpu().permute(1,2,0)+0.5)

In [ ]:
plt.imshow(output[0].cpu().permute(1,2,0)**.5*2)

In [ ]:
plt.imshow(output[1].cpu().permute(1,2,0)**.5*2)